# OANDA v20 API
## https://github.com/hootnot/oanda-api-v20/tree/master/oandapyV20

In [4]:
100 / 6

16.666666666666668

# FULL ALGO

In [1]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd


import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 

#import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import numpy as np
import datetime
from scipy import stats
import json
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

def convert_to_simple_df(instrument,first = True,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    #print(instrument,granularity,count)
    if first:
        params = {"count":count,
              "granularity": granularity
                 }
    else:
        params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }

    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')   
   # print('ACTUAL SHAPE:',df.shape,' MIN UTC:',df['utc_timestamp'].min(),' MAX UTC:',df['utc_timestamp'].max())
    return df
def add_feature_cols(df):
    
    def smma(df,period = 14):
        close_col = df.columns.get_loc('c')
        df['smma_' + str(period)] = df['c']
        smma_col = df.columns.get_loc('smma_' + str(period))
        arr = df.values

        #SMMA CALC:

        #SUM1=SUM (CLOSE, N)

        #SMMA1 = SUM1/ N

        #The second and subsequent moving averages are calculated according to this formula:

        #SMMA (i) = (SUM1 – SMMA1+CLOSE (i))/ N    

       # Where:

        #SUM1 – is the total sum of closing prices for N periods;
        #SMMA1 – is the smoothed moving average of the first bar;
        #SMMA (i) – is the smoothed moving average of the current bar (except the first one);
        #CLOSE (i) – is the current closing price;
        #N – is the smoothing period.    

        for i in range(period,arr.shape[0]):
            if i == period:
                sum1 = sum(arr[:i,close_col])

                smma1 = sum1 / period

                arr[i,smma_col] = smma1
            elif i == period + 1:
                arr[i,smma_col] = (smma1 * (period - 1) + arr[i,close_col]) / period

            else:

                prev_sum = arr[i - 1,smma_col] * period
                arr[i,smma_col] = (prev_sum - arr[i - 1,smma_col] + arr[i,close_col]) / period


        return pd.DataFrame(arr,columns = df.columns)    
    s = time.time()
    
    df['o'] = df['o'].astype(float)
    df['h'] = df['h'].astype(float)
    df['l'] = df['l'].astype(float)
    df['c'] = df['c'].astype(float)
    
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)

    df['delta_vector'] = 0

    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')

    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    e = time.time()
   # print('ADDING FEATURE COLS TIME:',round(e-s,3),'SECONDS')
    return df



def create_data(instrument,data_segments,count,pair,hours = 3,granularity = "M1"):

 #   print('CREATING DATASET...')
    start_ = time.time()
  #  print('PARAMS:',pair,granularity)

    total_df = convert_to_simple_df(instrument = instrument,first = True,granularity = granularity,count = count,start = datetime.datetime(2023,1,1,0,0,0),end = datetime.datetime(2023,1,1,0,0,0))
    cur = total_df['utc_timestamp'].iloc[0]
    df_lst = [total_df]
    for i in range(data_segments):
        df = convert_to_simple_df(instrument = instrument,first = False,granularity = granularity,count = count,start = cur - timedelta(hours = hours),end = cur)
        cur = cur - timedelta(hours = hours)
        df_lst.append(df)
       # total_df = pd.concat([total_df,df],axis = 0)
        #total_df = total_df.append(df)
    
    for item in enumerate(reversed(df_lst)):
        if item[0] == 0:
            total_df = item[1]
        else:
            total_df = pd.concat([total_df,item[1]],axis = 0)
    total_df = add_feature_cols(total_df)
   # print(total_df.shape)
#
    end_ = time.time()
   # print('TOTAL DATASET CONSTRUCTION TIME:',round(end_-start_,3),' SECONDS')
    return total_df



def signal_check(df,
                lookup_range = 1500,
                bullish_ma = True,
                bullish_candle = True,
                candle_size = .001,
                delta_filter = .001,
                delta_filter2 = .006,
                upper_wick = .0001,
                lower_wick = .0001,
                support = True                
                ):
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    ma1_col = df.columns.get_loc('smma_21')
    ma2_col = df.columns.get_loc('smma_50')
    ma3_col = df.columns.get_loc('smma_200')  
     
    
    
    i = df.shape[0] - 1
    #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
    max_ = df.iloc[i - lookup_range : i,c_col].max()
    min_ = df.iloc[i - lookup_range : i,c_col].min()
    resistance_delta = max_ - df.iloc[i,c_col] #arr[i,c_col]
    support_delta = df.iloc[i,c_col] - min_ #arr[i,c_col] - min_ 
    pip_range = max_ - min_    
    if bullish_ma == True:
        if df.iloc[i,ma1_col] > df.iloc[i,ma2_col] and df.iloc[i,ma2_col] > df.iloc[i,ma3_col]:
            ma_indicator = 1
        else:
            ma_indicator = 0
    elif bullish_ma == False:
        if df.iloc[i,ma1_col] < df.iloc[i,ma2_col] and df.iloc[i,ma2_col] < df.iloc[i,ma3_col]:
            ma_indicator = 1
        else:
            ma_indicator = 0      
            
            
    #CANDLE BULL OR BEAR
    if df.iloc[i,c_col] >= df.iloc[i,o_col]:
        bull = True
    else:
        bull = False      

    #Within the delta range:
    if support == True:
        if support_delta >= delta_filter and support_delta <= delta_filter + delta_filter2:
            near_reversal_indicator = 1
        else:
            near_reversal_indicator = 0
    else:
        if resistance_delta >= delta_filter and resistance_delta <= delta_filter + delta_filter2:
            near_reversal_indicator = 1
        else:
            near_reversal_indicator = 0            

    #within support and resistance range:
    if delta_filter < 0:
        if df.iloc[i,c_col] >= min_ - abs(delta_filter) and df.iloc[i,c_col] <= max_ + abs(delta_filter):
            between_s_and_r = 1
        else:
            between_s_and_r = 0            
    else:
        if df.iloc[i,c_col] >= min_ and df.iloc[i,c_col] <= max_:
            between_s_and_r = 1
        else:
            between_s_and_r = 0

    if between_s_and_r and \
    near_reversal_indicator == 1 and \
    ma_indicator == 1 and \
    df.iloc[i,d_col] >= candle_size and \
    df.iloc[i,uw_col] >= upper_wick and \
    df.iloc[i,lw_col] >= lower_wick and \
    bull == bullish_candle:
        return 1
    else:
        return 0 
def check_all_pairs(all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0):
    lst = []
    for pair in all_pairs:

        df = create_data(instrument = pair,data_segments = data_segments,count = count,pair = pair,hours = hours,granularity = "M5")
        last_utc = df['utc_timestamp'].iloc[-1]
        last_est = df['est_timestamp'].iloc[-1]
        last_close = df['c'].iloc[-1]

        
        #print(df.shape)    
        buy_or_sell = 'buy'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    

        buy_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
        buy_or_sell = 'sell'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    
        

        sell_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
      #  print(buy_signal,sell_signal)
      #  if buy_signal == 1 and placed_buy_in_last_24_candles == 0:
       #     print('BUYING')
        #if sell_signal == 1 and placed_sell_in_last_24_candles == 0:
         #   print('SELLING')
        #if buy_signal + sell_signal == 0 or placed_buy_in_last_24_candles + placed_sell_in_last_24_candles != 0:
         #   print('NOT PLACING TRADE')
            
        lst.append([pair,str_time,last_utc,last_est,last_close,buy_signal,sell_signal])
        
        
    return pd.DataFrame(lst,columns = ['pair','sys_time_5_min','last_utc','last_est','last_close','buy_signal','sell_signal'])
def get_account_balance():
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    bal = acc_details['account']['balance']
    margin_available = acc_details['account']['marginAvailable']
    print('CURRENT BALANCE:',round(float(bal)),'CURRENT MARGIN AVAILABLE',round(float(margin_available)))
    return min(float(bal),float(margin_available))

def units_calc(trading_account,max_risk,sl):
    units=round((max_risk * trading_account / sl) * 10000)
    print('MAX RISK:',max_risk,'SL:',sl,'LOTS:',round(units / 100000,2),'UNITS:',units)
    return units
def create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS):
    def check_price(pair):
        api = oandapyV20.API(access_token=access_token)
        bid_ask=[]
        params={"instruments":pair}

        r = pricing.PricingInfo(accountID=accountID, params=params)
        a = api.request(r)

        bid=float(a['prices'][0]['bids'][0]['price'])
        ask=float(a['prices'][0]['asks'][0]['price'])
        avg=(bid+ask)/2

        bid_ask.append(bid)
        bid_ask.append(ask)
        bid_ask.append(avg)



        return bid_ask
    
    if 'JPY' in instrument:
        multiplier = .01
    else:
        multiplier = .0001
    if units > 0:
        price = min(check_price(instrument))
        TAKE_PROFIT = round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price - (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (STOP_LOSS_NUM_PIPS * multiplier),5)
    else:
        price = max(check_price(instrument))
        TAKE_PROFIT = round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price + (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (STOP_LOSS_NUM_PIPS * multiplier),5)
        
        
    print('CREATING MARKET ORDER:',instrument,'|LOTS:',units / 100000,'|UNITS:',units,'|TP NUM PIPS:',TAKE_PROFIT_NUM_PIPS,
          '|SL NUM PIPS:',STOP_LOSS_NUM_PIPS,'|CUR PRICE:',price,'|STOP LOSS:',STOP_LOSS,'|TAKE PROFIT:',TAKE_PROFIT)
    print()
    
    api = oandapyV20.API(access_token=access_token)

    mktOrder = MarketOrderRequest(
        instrument = instrument,
        units = units,
        takeProfitOnFill=TakeProfitDetails(price=TAKE_PROFIT).data,
        stopLossOnFill=StopLossDetails(price=STOP_LOSS).data)

    # create the OrderCreate request
    r = orders.OrderCreate(accountID, data=mktOrder.data)
    try:
        # create the OrderCreate request
        rv = api.request(r)
    except oandapyV20.exceptions.V20Error as err:
        print(r.status_code, err)
    else:
        print(json.dumps(rv, indent=2))
        
    return rv

def enter_trade(dir_name,param_df,pair,buy_or_sell):
    import json
    # get stop loss and take profit
    sl = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['sl'].iloc[0]
    tp = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['tp'].iloc[0]


    # get balance and estimated units to trade
    if 'JPY' in pair:
        TAKE_PROFIT_NUM_PIPS = tp * 100
        STOP_LOSS_NUM_PIPS = sl * 100
    else:
        TAKE_PROFIT_NUM_PIPS = tp * 100 * 100
        STOP_LOSS_NUM_PIPS = sl * 100 * 100

    balance = get_account_balance()
    units = units_calc(trading_account = balance,max_risk = .01,sl = STOP_LOSS_NUM_PIPS)

    instrument =  pair
    if buy_or_sell != 'buy':
        units = units * -1
        
    order_dict = create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS)

    new_order_dict = json.dumps(order_dict)
    # open file for writing, "w" 
    dict_file = dir_name + "/order_dict_" + pair + '_' + buy_or_sell+ '_' + str(round(units /100000)) + '_' + str(TAKE_PROFIT_NUM_PIPS) + '_' + str(STOP_LOSS_NUM_PIPS) + ".json"
    print('SAVING ORDER DICT:',dict_file)
    f = open(dict_file,"w")

    # write json object to file
    f.write(new_order_dict)

    # close file
    f.close()
def run_trade_algo_once(dir_name):
    def run_trade_if_available(dir_name,check_df,param_df):

        buys = list(check_df[check_df['buy_signal'] == 1]['pair'])
        sells = list(check_df[check_df['buy_signal'] == 1]['pair'])
        print('BUY LEN:',len(buys),'SELL LEN:',len(sells))
        for pair in buys:
            print(pair)
            enter_trade(dir_name,param_df,pair,buy_or_sell = 'buy')
        for pair in sells:
            print(pair)
            enter_trade(dir_name,param_df,pair,buy_or_sell = 'sell')      
    s = time.time()

    trade_check_path = dir_name + '/trade_check_data.csv'
    cur_time = datetime.datetime.now()
    str_time = str(cur_time.year) + '_' + str(cur_time.month) + '_' + str(cur_time.day) + '_' + str(cur_time.hour) + '_' + str((cur_time.minute // 5) * 5)
    print('CUR TIME:',cur_time,str_time)
    try:
        trade_check_df = pd.read_csv(trade_check_path)

        if str_time in list(trade_check_df['sys_time_5_min']):
            print(str_time,'ALREADY EXISTS, PASSING',cur_time.minute - ((cur_time.minute//5) * 5))
            if cur_time.minute - ((cur_time.minute//5) * 5) in (0,1,2,3):
                time.sleep(60)
            else:
                time.sleep(30)
        else:
            print('DOES NOT EXIST, RUNNING TRADE CHECKER')
            check_df = check_all_pairs(all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 

            trade_check_df = pd.concat([trade_check_df,check_df],axis = 0)
            trade_check_df.to_csv(trade_check_path,index = False)
            run_trade_if_available(dir_name,check_df,param_df)

    except FileNotFoundError:
        print("FileNotFoundError: running loop once to get file")
        check_df = check_all_pairs(all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 
        check_df.to_csv(trade_check_path,index = False)
        run_trade_if_available(dir_name,check_df,param_df)

    e = time.time()
    print((e-s),'TOTAL SECONDS') 
    
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

file = 'best_trade_params.csv'
param_df = pd.read_csv(file)
count = 5000
data_segments = 5
hours = 250

dir_name = os.getcwd() + '/trading_folder'

isdir = os.path.isdir(dir_name) 
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name) 
today = date.today()
dir_name = os.getcwd() + '/trading_folder/' + str(today)

isdir = os.path.isdir(dir_name) 
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name)     
    
    
while 1 == 1:    
    run_trade_algo_once(dir_name)


    
    
    

CUR TIME: 2023_2_20_9_50
2023_2_20_9_50 ALREADY EXISTS, PASSING 4
30.005409002304077 TOTAL SECONDS
CUR TIME: 2023_2_20_9_55
DOES NOT EXIST, RUNNING TRADE CHECKER
BUY LEN: 0 SELL LEN: 0
19.15474510192871 TOTAL SECONDS
CUR TIME: 2023_2_20_9_55
2023_2_20_9_55 ALREADY EXISTS, PASSING 0
60.00698232650757 TOTAL SECONDS
CUR TIME: 2023_2_20_9_55
2023_2_20_9_55 ALREADY EXISTS, PASSING 1
60.022526025772095 TOTAL SECONDS
CUR TIME: 2023_2_20_9_55
2023_2_20_9_55 ALREADY EXISTS, PASSING 2
60.01833987236023 TOTAL SECONDS
CUR TIME: 2023_2_20_9_55
2023_2_20_9_55 ALREADY EXISTS, PASSING 3
60.00983715057373 TOTAL SECONDS
CUR TIME: 2023_2_20_9_55
2023_2_20_9_55 ALREADY EXISTS, PASSING 4
30.008763313293457 TOTAL SECONDS
CUR TIME: 2023_2_20_9_55
2023_2_20_9_55 ALREADY EXISTS, PASSING 4
30.006764888763428 TOTAL SECONDS
CUR TIME: 2023_2_20_10_0
DOES NOT EXIST, RUNNING TRADE CHECKER
BUY LEN: 0 SELL LEN: 0
20.854460954666138 TOTAL SECONDS
CUR TIME: 2023_2_20_10_0
2023_2_20_10_0 ALREADY EXISTS, PASSING 0
60.0073

KeyboardInterrupt: 

In [5]:
def units_calc(trading_account,max_risk,sl):
    units=round((max_risk * trading_account / sl) * 10000)
    print('MAX RISK:',max_risk,'SL:',sl,'LOTS:',round(units / 100000,2),'UNITS:',units)
    return units
trading_account = 100000
max_risk = .01
sl = 10
units_calc(trading_account,max_risk,sl)

MAX RISK: 0.01 SL: 10 LOTS: 10.0 UNITS: 1000000


1000000

In [15]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd


import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 

#import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import numpy as np
import datetime
from scipy import stats
import json

from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

def convert_to_simple_df(instrument,first = True,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    #print(instrument,granularity,count)
    if first:
        params = {"count":count,
              "granularity": granularity
                 }
    else:
        params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }

    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    
    access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'
    
    api = API(access_token)
    client=API(access_token)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')   
   # print('ACTUAL SHAPE:',df.shape,' MIN UTC:',df['utc_timestamp'].min(),' MAX UTC:',df['utc_timestamp'].max())
    return df
def add_feature_cols(df):
    
    def smma(df,period = 14):
        close_col = df.columns.get_loc('c')
        df['smma_' + str(period)] = df['c']
        smma_col = df.columns.get_loc('smma_' + str(period))
        arr = df.values

        #SMMA CALC:

        #SUM1=SUM (CLOSE, N)

        #SMMA1 = SUM1/ N

        #The second and subsequent moving averages are calculated according to this formula:

        #SMMA (i) = (SUM1 – SMMA1+CLOSE (i))/ N    

       # Where:

        #SUM1 – is the total sum of closing prices for N periods;
        #SMMA1 – is the smoothed moving average of the first bar;
        #SMMA (i) – is the smoothed moving average of the current bar (except the first one);
        #CLOSE (i) – is the current closing price;
        #N – is the smoothing period.    

        for i in range(period,arr.shape[0]):
            if i == period:
                sum1 = sum(arr[:i,close_col])

                smma1 = sum1 / period

                arr[i,smma_col] = smma1
            elif i == period + 1:
                arr[i,smma_col] = (smma1 * (period - 1) + arr[i,close_col]) / period

            else:

                prev_sum = arr[i - 1,smma_col] * period
                arr[i,smma_col] = (prev_sum - arr[i - 1,smma_col] + arr[i,close_col]) / period


        return pd.DataFrame(arr,columns = df.columns)    
    s = time.time()
    
    df['o'] = df['o'].astype(float)
    df['h'] = df['h'].astype(float)
    df['l'] = df['l'].astype(float)
    df['c'] = df['c'].astype(float)
    
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)

    df['delta_vector'] = 0

    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')

    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    e = time.time()
   # print('ADDING FEATURE COLS TIME:',round(e-s,3),'SECONDS')
    return df



def create_data(instrument,data_segments,count,pair,hours = 3,granularity = "M1"):

 #   print('CREATING DATASET...')
    start_ = time.time()
  #  print('PARAMS:',pair,granularity)

    total_df = convert_to_simple_df(instrument = instrument,first = True,granularity = granularity,count = count,start = datetime.datetime(2023,1,1,0,0,0),end = datetime.datetime(2023,1,1,0,0,0))
    cur = total_df['utc_timestamp'].iloc[0]
    df_lst = [total_df]
    for i in range(data_segments):
        df = convert_to_simple_df(instrument = instrument,first = False,granularity = granularity,count = count,start = cur - timedelta(hours = hours),end = cur)
        cur = cur - timedelta(hours = hours)
        df_lst.append(df)
       # total_df = pd.concat([total_df,df],axis = 0)
        #total_df = total_df.append(df)
    
    for item in enumerate(reversed(df_lst)):
        if item[0] == 0:
            total_df = item[1]
        else:
            total_df = pd.concat([total_df,item[1]],axis = 0)
    total_df = add_feature_cols(total_df)
   # print(total_df.shape)
#
    end_ = time.time()
   # print('TOTAL DATASET CONSTRUCTION TIME:',round(end_-start_,3),' SECONDS')
    return total_df



def signal_check(df,
                lookup_range = 1500,
                bullish_ma = True,
                bullish_candle = True,
                candle_size = .001,
                delta_filter = .001,
                delta_filter2 = .006,
                upper_wick = .0001,
                lower_wick = .0001,
                support = True                
                ):
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    ma1_col = df.columns.get_loc('smma_21')
    ma2_col = df.columns.get_loc('smma_50')
    ma3_col = df.columns.get_loc('smma_200')  
     
    
    
    i = df.shape[0] - 1
    #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
    max_ = df.iloc[i - lookup_range : i,c_col].max()
    min_ = df.iloc[i - lookup_range : i,c_col].min()
    resistance_delta = max_ - df.iloc[i,c_col] #arr[i,c_col]
    support_delta = df.iloc[i,c_col] - min_ #arr[i,c_col] - min_ 
    pip_range = max_ - min_    
    if bullish_ma == True:
        if df.iloc[i,ma1_col] > df.iloc[i,ma2_col] and df.iloc[i,ma2_col] > df.iloc[i,ma3_col]:
            ma_indicator = 1
        else:
            ma_indicator = 0
    elif bullish_ma == False:
        if df.iloc[i,ma1_col] < df.iloc[i,ma2_col] and df.iloc[i,ma2_col] < df.iloc[i,ma3_col]:
            ma_indicator = 1
        else:
            ma_indicator = 0      
            
            
    #CANDLE BULL OR BEAR
    if df.iloc[i,c_col] >= df.iloc[i,o_col]:
        bull = True
    else:
        bull = False      

    #Within the delta range:
    if support == True:
        if support_delta >= delta_filter and support_delta <= delta_filter + delta_filter2:
            near_reversal_indicator = 1
        else:
            near_reversal_indicator = 0
    else:
        if resistance_delta >= delta_filter and resistance_delta <= delta_filter + delta_filter2:
            near_reversal_indicator = 1
        else:
            near_reversal_indicator = 0            

    #within support and resistance range:
    if delta_filter < 0:
        if df.iloc[i,c_col] >= min_ - abs(delta_filter) and df.iloc[i,c_col] <= max_ + abs(delta_filter):
            between_s_and_r = 1
        else:
            between_s_and_r = 0            
    else:
        if df.iloc[i,c_col] >= min_ and df.iloc[i,c_col] <= max_:
            between_s_and_r = 1
        else:
            between_s_and_r = 0

    if between_s_and_r and \
    near_reversal_indicator == 1 and \
    ma_indicator == 1 and \
    df.iloc[i,d_col] >= candle_size and \
    df.iloc[i,uw_col] >= upper_wick and \
    df.iloc[i,lw_col] >= lower_wick and \
    bull == bullish_candle:
        return 1
    else:
        return 0 
def check_all_pairs(param_df,all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0):
    lst = []
    count = 5000
    data_segments = 5
    hours = 250
    for pair in all_pairs:

        df = create_data(instrument = pair,data_segments = data_segments,count = count,pair = pair,hours = hours,granularity = "M5")
        last_utc = df['utc_timestamp'].iloc[-1]
        last_est = df['est_timestamp'].iloc[-1]
        last_close = df['c'].iloc[-1]

        
        #print(df.shape)    
        buy_or_sell = 'buy'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    

        buy_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
        buy_or_sell = 'sell'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    
        

        sell_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
      #  print(buy_signal,sell_signal)
      #  if buy_signal == 1 and placed_buy_in_last_24_candles == 0:
       #     print('BUYING')
        #if sell_signal == 1 and placed_sell_in_last_24_candles == 0:
         #   print('SELLING')
        #if buy_signal + sell_signal == 0 or placed_buy_in_last_24_candles + placed_sell_in_last_24_candles != 0:
         #   print('NOT PLACING TRADE')
            
        lst.append([pair,str_time,last_utc,last_est,last_close,buy_signal,sell_signal])
        
        
    return pd.DataFrame(lst,columns = ['pair','sys_time_5_min','last_utc','last_est','last_close','buy_signal','sell_signal'])
def get_account_balance():
    access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'
    accountID='101-001-20062555-002'
    
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    bal = acc_details['account']['balance']
    margin_available = acc_details['account']['marginAvailable']
    print('CURRENT BALANCE:',round(float(bal)),'CURRENT MARGIN AVAILABLE',round(float(margin_available)))
    return min(float(bal),float(margin_available))

def units_calc(trading_account,max_risk,sl):
    units=round((max_risk * trading_account / sl) * 10000)
    print('MAX RISK:',max_risk,'SL:',sl,'LOTS:',round(units / 100000,2),'UNITS:',units)
    return units
def create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS):
    def check_price(pair):
        access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'
        accountID='101-001-20062555-002'
        api = oandapyV20.API(access_token=access_token)
        bid_ask=[]
        params={"instruments":pair}

        r = pricing.PricingInfo(accountID=accountID, params=params)
        a = api.request(r)

        bid=float(a['prices'][0]['bids'][0]['price'])
        ask=float(a['prices'][0]['asks'][0]['price'])
        avg=(bid+ask)/2

        bid_ask.append(bid)
        bid_ask.append(ask)
        bid_ask.append(avg)



        return bid_ask
    
    if 'JPY' in instrument:
        multiplier = .01
    else:
        multiplier = .0001
    if units > 0:
        price = min(check_price(instrument))
        TAKE_PROFIT = round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price - (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (STOP_LOSS_NUM_PIPS * multiplier),5)
    else:
        price = max(check_price(instrument))
        TAKE_PROFIT = round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price + (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (STOP_LOSS_NUM_PIPS * multiplier),5)
        
        
    print('CREATING MARKET ORDER:',instrument,'|LOTS:',units / 100000,'|UNITS:',units,'|TP NUM PIPS:',TAKE_PROFIT_NUM_PIPS,
          '|SL NUM PIPS:',STOP_LOSS_NUM_PIPS,'|CUR PRICE:',price,'|STOP LOSS:',STOP_LOSS,'|TAKE PROFIT:',TAKE_PROFIT)
    print()
    access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'
    accountID='101-001-20062555-002'    
    api = oandapyV20.API(access_token=access_token)

    mktOrder = MarketOrderRequest(
        instrument = instrument,
        units = units,
        takeProfitOnFill=TakeProfitDetails(price=TAKE_PROFIT).data,
        stopLossOnFill=StopLossDetails(price=STOP_LOSS).data)

    # create the OrderCreate request
    r = orders.OrderCreate(accountID, data=mktOrder.data)
    try:
        # create the OrderCreate request
        rv = api.request(r)
    except oandapyV20.exceptions.V20Error as err:
        print(r.status_code, err)
    else:
        print(json.dumps(rv, indent=2))
        
    return rv

def enter_trade(dir_name,param_df,pair,buy_or_sell):
    import json
    # get stop loss and take profit
    sl = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['sl'].iloc[0]
    tp = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['tp'].iloc[0]


    # get balance and estimated units to trade
    if 'JPY' in pair:
        TAKE_PROFIT_NUM_PIPS = tp * 100
        STOP_LOSS_NUM_PIPS = sl * 100
    else:
        TAKE_PROFIT_NUM_PIPS = tp * 100 * 100
        STOP_LOSS_NUM_PIPS = sl * 100 * 100

    balance = get_account_balance()
    units = units_calc(trading_account = balance,max_risk = .01,sl = STOP_LOSS_NUM_PIPS)

    instrument =  pair
    if buy_or_sell != 'buy':
        units = units * -1
        
    order_dict = create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS)

    new_order_dict = json.dumps(order_dict)
    # open file for writing, "w" 
    dict_file = dir_name + "/order_dict_" + pair + '_' + buy_or_sell+ '_' + str(round(units /100000)) + '_' + str(TAKE_PROFIT_NUM_PIPS) + '_' + str(STOP_LOSS_NUM_PIPS) + ".json"
    print('SAVING ORDER DICT:',dict_file)
    f = open(dict_file,"w")

    # write json object to file
    f.write(new_order_dict)

    # close file
    f.close()
def run_trade_algo_once(dir_name,param_df):
    def run_trade_if_available(dir_name,check_df,param_df):

        buys = list(check_df[check_df['buy_signal'] == 1]['pair'])
        sells = list(check_df[check_df['buy_signal'] == 1]['pair'])
        print('BUY LEN:',len(buys),'SELL LEN:',len(sells))
        for pair in buys:
            print(pair)
            enter_trade(dir_name,param_df,pair,buy_or_sell = 'buy')
        for pair in sells:
            print(pair)
            enter_trade(dir_name,param_df,pair,buy_or_sell = 'sell')      
    s = time.time()
    all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

    trade_check_path = dir_name + '/trade_check_data.csv'
    cur_time = datetime.datetime.now()
    str_time = str(cur_time.year) + '_' + str(cur_time.month) + '_' + str(cur_time.day) + '_' + str(cur_time.hour) + '_' + str((cur_time.minute // 5) * 5)
    print('CUR TIME:',cur_time,str_time)
    try:
        trade_check_df = pd.read_csv(trade_check_path)

        if str_time in list(trade_check_df['sys_time_5_min']):
            seconds_at_start_of_next_candle = (((cur_time.minute + 5)//5) * 5) * 60
            second_right_now = (cur_time.minute * 60) + cur_time.second
            sleep_seconds = seconds_at_start_of_next_candle - second_right_now
            print(str_time,'ALREADY EXISTS, PASSING',seconds_at_start_of_next_candle , second_right_now,sleep_seconds)
            time.sleep(sleep_seconds)
        else:
            print('DOES NOT EXIST, RUNNING TRADE CHECKER')
            check_df = check_all_pairs(param_df,all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 

            trade_check_df = pd.concat([trade_check_df,check_df],axis = 0)
            trade_check_df.to_csv(trade_check_path,index = False)
            run_trade_if_available(dir_name,check_df,param_df)

    except FileNotFoundError:
        print("FileNotFoundError: running loop once to get file")
        check_df = check_all_pairs(param_df,all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 
        check_df.to_csv(trade_check_path,index = False)
        run_trade_if_available(dir_name,check_df,param_df)

    e = time.time()
    print((e-s),'TOTAL SECONDS') 
    
    
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)




file = 'best_trade_params.csv'
param_df = pd.read_csv(file)


dir_name = os.getcwd() + '/trading_folder'

isdir = os.path.isdir(dir_name) 
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name) 

    
    



today = date.today()
dir_name = os.getcwd() + '/trading_folder/' + str(today)

isdir = os.path.isdir(dir_name)
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name)
s = time.time()
all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
       'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
       'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
       'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

trade_check_path = dir_name + '/trade_check_data.csv'
cur_time = datetime.datetime.now()
str_time = str(cur_time.year) + '_' + str(cur_time.month) + '_' + str(cur_time.day) + '_' + str(cur_time.hour) + '_' + str((cur_time.minute // 5) * 5)
print('CUR TIME:',cur_time,str_time)
try:
    trade_check_df = pd.read_csv(trade_check_path)

    if str_time in list(trade_check_df['sys_time_5_min']):
        seconds_at_start_of_next_candle = (((cur_time.minute + 5)//5) * 5) * 60
        second_right_now = (cur_time.minute * 60) + cur_time.second
        sleep_seconds = seconds_at_start_of_next_candle - second_right_now
        print(str_time,'ALREADY EXISTS, PASSING',seconds_at_start_of_next_candle , second_right_now,sleep_seconds)
        time.sleep(sleep_seconds)
    else:
        print('DOES NOT EXIST, RUNNING TRADE CHECKER')
        check_df = check_all_pairs(param_df,all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 

        trade_check_df = pd.concat([trade_check_df,check_df],axis = 0)
        trade_check_df.to_csv(trade_check_path,index = False)


except FileNotFoundError:
    print("FileNotFoundError: running loop once to get file")
    check_df = check_all_pairs(param_df,all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 
    check_df.to_csv(trade_check_path,index = False)


e = time.time()
print((e-s),'TOTAL SECONDS') 
print(check_df.shape)
check_df.iloc[1,-1] = 1
check_df.iloc[2,-2] = 1
check_df.head()  


CUR TIME: 2023-02-23 08:15:37.507852 2023_2_23_8_15
DOES NOT EXIST, RUNNING TRADE CHECKER
26.335163116455078 TOTAL SECONDS
(21, 7)


,pair,sys_time_5_min,last_utc,last_est,last_close,buy_signal,sell_signal
0,GBP_JPY,2023_2_23_8_15,2023-02-23 14:15:00+00:00,2023-02-23 09:15:00-05:00,162.80800,0,0
1,EUR_USD,2023_2_23_8_15,2023-02-23 14:15:00+00:00,2023-02-23 09:15:00-05:00,1.06108,0,1
2,USD_JPY,2023_2_23_8_15,2023-02-23 14:15:00+00:00,2023-02-23 09:15:00-05:00,135.19400,1,0
3,EUR_JPY,2023_2_23_8_15,2023-02-23 14:15:00+00:00,2023-02-23 09:15:00-05:00,143.45400,0,0
4,AUD_JPY,2023_2_23_8_15,2023-02-23 14:15:00+00:00,2023-02-23 09:15:00-05:00,92.24400,0,0


In [13]:
buys = list(check_df[check_df['buy_signal'] == 1]['pair'])
sells = list(check_df[check_df['buy_signal'] == 1]['pair'])
print('BUY LEN:',len(buys),'SELL LEN:',len(sells))

BUY LEN: 1 SELL LEN: 1


In [24]:
def run_trade_if_available(dir_name,check_df,param_df):

    buys = list(check_df[check_df['buy_signal'] == 1]['pair'])
    sells = list(check_df[check_df['sell_signal'] == 1]['pair'])
    print('BUY LEN:',len(buys),'SELL LEN:',len(sells))
    for pair in buys:
        print(pair)
        enter_trade(dir_name,param_df,pair,buy_or_sell = 'buy')
    for pair in sells:
        print(pair)
        enter_trade(dir_name,param_df,pair,buy_or_sell = 'sell') 
        
run_trade_if_available(dir_name,check_df,param_df)       

BUY LEN: 1 SELL LEN: 1
USD_JPY
CURRENT BALANCE: 95327 CURRENT MARGIN AVAILABLE 95327
MAX RISK: 0.01 SL: 5.0 LOTS: 19.07 UNITS: 1906532
CREATING MARKET ORDER: USD_JPY |LOTS: 19.06532 |UNITS: 1906532 |TP NUM PIPS: 70.0 |SL NUM PIPS: 5.0 |CUR PRICE: 135.101 |STOP LOSS: 135.051 |TAKE PROFIT: 135.801

{
  "orderCreateTransaction": {
    "id": "258",
    "accountID": "101-001-20062555-002",
    "userID": 20062555,
    "batchID": "258",
    "requestID": "25048925763385734",
    "time": "2023-02-23T14:19:52.508928640Z",
    "type": "MARKET_ORDER",
    "instrument": "USD_JPY",
    "units": "1906532",
    "timeInForce": "FOK",
    "positionFill": "DEFAULT",
    "takeProfitOnFill": {
      "price": "135.801",
      "timeInForce": "GTC"
    },
    "stopLossOnFill": {
      "price": "135.051",
      "timeInForce": "GTC",
      "triggerMode": "TOP_OF_BOOK"
    },
    "reason": "CLIENT_ORDER"
  },
  "orderFillTransaction": {
    "id": "259",
    "accountID": "101-001-20062555-002",
    "userID": 2006

In [ ]:
pair = 'GBP_JPY'

buy_or_sell = 'buy'

buys = list(check_df[check_df['buy_signal'] == 1]['pair'])
sells = list(check_df[check_df['buy_signal'] == 1]['pair'])
print('BUY LEN:',len(buys),'SELL LEN:',len(sells))
for pair in buys:
    print(pair)
    enter_trade(param_df,pair,buy_or_sell = 'buy')
for pair in sells:
    print(pair)
    enter_trade(param_df,pair,buy_or_sell = 'sell')

In [298]:

def enter_trade(dir_name,param_df,pair,buy_or_sell):
    import json
    # get stop loss and take profit
    sl = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['sl'].iloc[0]
    tp = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['tp'].iloc[0]


    # get balance and estimated units to trade
    if 'JPY' in pair:
        TAKE_PROFIT_NUM_PIPS = tp * 100
        STOP_LOSS_NUM_PIPS = sl * 100
    else:
        TAKE_PROFIT_NUM_PIPS = tp * 100 * 100
        STOP_LOSS_NUM_PIPS = sl * 100 * 100

    balance = get_account_balance()
    units = units_calc(trading_account = balance,max_risk = .01,sl = int_sl)

    instrument =  pair
    if buy_or_sell != 'buy':
        units = units * -1
        
    order_dict = create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS)

    new_order_dict = json.dumps(order_dict)
    # open file for writing, "w" 
    dict_file = dir_name + "/order_dict_" + pair + '_' + buy_or_sell+ '_' + str(round(units /100000)) + '_' + str(TAKE_PROFIT_NUM_PIPS) + '_' + str(STOP_LOSS_NUM_PIPS) + ".json"
    print('SAVING ORDER DICT:',dict_file)
    f = open(dict_file,"w")

    # write json object to file
    f.write(new_order_dict)

    # close file
    f.close()
    
pair = 'GBP_JPY'
order_dict = enter_trade(dir_name,param_df,pair,buy_or_sell = 'buy')
order_dict

CURRENT BALANCE: 96089 CURRENT MARGIN AVAILABLE 96089
MAX RISK: 0.01 SL: 5.0 LOTS: 19.22 UNITS: 1921788
CREATING MARKET ORDER: GBP_JPY |LOTS: 19.21788 |UNITS: 1921788 |TP NUM PIPS: 90.0 |SL NUM PIPS: 5.0 |CUR PRICE: 161.205 |STOP LOSS: 161.155 |TAKE PROFIT: 162.105

{
  "orderCreateTransaction": {
    "id": "242",
    "accountID": "101-001-20062555-002",
    "userID": 20062555,
    "batchID": "242",
    "requestID": "43062250561858872",
    "time": "2023-02-20T15:13:19.240063261Z",
    "type": "MARKET_ORDER",
    "instrument": "GBP_JPY",
    "units": "1921788",
    "timeInForce": "FOK",
    "positionFill": "DEFAULT",
    "takeProfitOnFill": {
      "price": "162.105",
      "timeInForce": "GTC"
    },
    "stopLossOnFill": {
      "price": "161.155",
      "timeInForce": "GTC",
      "triggerMode": "TOP_OF_BOOK"
    },
    "reason": "CLIENT_ORDER"
  },
  "orderFillTransaction": {
    "id": "243",
    "accountID": "101-001-20062555-002",
    "userID": 20062555,
    "batchID": "242",
   

In [292]:
type(order_dict)

dict

In [314]:
((25 * 5) * 5) / 60

10.416666666666666

In [28]:
from oandapyV20.endpoints.positions import PositionList



def close_trade(trade_id):
    print('CLOSING TRADE:',trade_id)
    api = oandapyV20.API(access_token=access_token)
    r = trades.TradeClose(accountID = accountID,tradeID = trade_id)
    t = api.request(r)
    print(t)
api = oandapyV20.API(access_token=access_token)
r = positions.OpenPositions(accountID)
pos = api.request(r)
long_or_short = 'long'
close_trade(pos['positions'][0][long_or_short]['tradeIDs'][0])      

IndexError: list index out of range

In [29]:
pos

{'positions': [], 'lastTransactionID': '271'}

In [23]:
pos

{'positions': [], 'lastTransactionID': '257'}

In [22]:
pos['positions'][0]['short']

IndexError: list index out of range

In [307]:
from oandapyV20.endpoints.orders import OrderList
client = oandapyV20.API(access_token=access_token)

# Define the parameters for the order list endpoint
params = {"accountID": accountID}

# Call the order list endpoint to retrieve all the orders for the specified account
r = OrderList(accountID = accountID)
orders = client.request(r)
orders

{'orders': [], 'lastTransactionID': '249'}

In [317]:
from oandapyV20.endpoints.positions import PositionList

r = PositionList(accountID = accountID)
positions = client.request(r)

positions


{'positions': [{'instrument': 'GBP_JPY',
   'long': {'units': '0',
    'pl': '-2499.0595',
    'resettablePL': '-2499.0595',
    'financing': '0.0000',
    'dividendAdjustment': '0.0000',
    'guaranteedExecutionFees': '0.0000',
    'unrealizedPL': '0.0000'},
   'short': {'units': '0',
    'pl': '0.0000',
    'resettablePL': '0.0000',
    'financing': '0.0000',
    'dividendAdjustment': '0.0000',
    'guaranteedExecutionFees': '0.0000',
    'unrealizedPL': '0.0000'},
   'pl': '-2499.0595',
   'resettablePL': '-2499.0595',
   'financing': '0.0000',
   'commission': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  {'instrument': 'EUR_NZD',
   'long': {'units': '0',
    'pl': '0.2874',
    'resettablePL': '0.2874',
    'financing': '0.0000',
    'dividendAdjustment': '0.0000',
    'guaranteedExecutionFees': '0.0000',
    'unrealizedPL': '0.0000'},
   'short': {'units': '0',
    'pl': '0.0000',
    'resettablePL': '0.0000'

In [318]:
from oandapyV20.endpoints.trades import TradesList
client = oandapyV20.API(access_token=access_token)
r = TradesList(accountID = accountID)
orders = client.request(r)
orders

{'trades': [], 'lastTransactionID': '249'}

In [319]:
from oandapyV20.endpoints.transactions import TransactionList
client = oandapyV20.API(access_token=access_token)
r = TransactionList(accountID = accountID)
orders = client.request(r)
orders

{'from': '2023-02-13T15:33:09.213488024Z',
 'to': '2023-02-20T15:33:09.213488024Z',
 'pageSize': 100,
 'count': 136,
 'pages': ['https://api-fxpractice.oanda.com/v3/accounts/101-001-20062555-002/transactions/idrange?from=114&to=213',
  'https://api-fxpractice.oanda.com/v3/accounts/101-001-20062555-002/transactions/idrange?from=214&to=249'],
 'lastTransactionID': '249'}

In [202]:
dir_name = os.getcwd() + '/trading_folder'

isdir = os.path.isdir(dir_name) 
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name) 

MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/trading_folder


In [206]:
datetime.datetime.now()

datetime.datetime(2023, 2, 17, 17, 1, 5, 281602)

In [203]:
date.today()

datetime.date(2023, 2, 17)

In [205]:
def check_all_pairs(all_pairs):
    lst = []
    for pair in all_pairs:

        df = create_data(instrument = pair,data_segments = data_segments,count = count,pair = pair,hours = hours,granularity = "M5")
        last_utc = df['utc_timestamp'].iloc[-1]
        last_est = df['est_timestamp'].iloc[-1]
        last_close = df['c'].iloc[-1]

        placed_buy_in_last_24_candles = 0
        print(df.shape)    
        buy_or_sell = 'buy'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    

        buy_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
        buy_or_sell = 'sell'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    
        placed_sell_in_last_24_candles = 0

        sell_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
        print(buy_signal,sell_signal)
        if buy_signal == 1 and placed_buy_in_last_24_candles == 0:
            print('BUYING')
        if sell_signal == 1 and placed_sell_in_last_24_candles == 0:
            print('SELLING')
        if buy_signal + sell_signal == 0 or placed_buy_in_last_24_candles + placed_sell_in_last_24_candles != 0:
            print('NOT PLACING TRADE')
            
        lst.append([pair,last_utc,last_est,last_close,buy_signal,sell_signal])
        
        
    return pd.DataFrame(lst,columns = ['pair','last_utc','last_est','last_close','buy_signal','sell_signal'])
            
check_df = check_all_pairs(all_pairs)
check_df

CREATING DATASET...
PARAMS: GBP_JPY M5
ADDING FEATURE COLS TIME: 0.319 SECONDS
(15529, 15)
TOTAL DATASET CONSTRUCTION TIME: 0.904  SECONDS
(15529, 15)
0 0
NOT PLACING TRADE
CREATING DATASET...
PARAMS: EUR_USD M5
ADDING FEATURE COLS TIME: 0.301 SECONDS
(15507, 15)
TOTAL DATASET CONSTRUCTION TIME: 0.868  SECONDS
(15507, 15)
0 0
NOT PLACING TRADE
CREATING DATASET...
PARAMS: USD_JPY M5
ADDING FEATURE COLS TIME: 0.302 SECONDS
(15529, 15)
TOTAL DATASET CONSTRUCTION TIME: 0.88  SECONDS
(15529, 15)
0 0
NOT PLACING TRADE
CREATING DATASET...
PARAMS: EUR_JPY M5
ADDING FEATURE COLS TIME: 0.339 SECONDS
(15530, 15)
TOTAL DATASET CONSTRUCTION TIME: 0.88  SECONDS
(15530, 15)
0 0
NOT PLACING TRADE
CREATING DATASET...
PARAMS: AUD_JPY M5
ADDING FEATURE COLS TIME: 0.34 SECONDS
(15523, 15)
TOTAL DATASET CONSTRUCTION TIME: 0.858  SECONDS
(15523, 15)
0 0
NOT PLACING TRADE
CREATING DATASET...
PARAMS: CAD_JPY M5
ADDING FEATURE COLS TIME: 0.3 SECONDS
(15532, 15)
TOTAL DATASET CONSTRUCTION TIME: 0.843  SECONDS
(

,pair,last_utc,last_est,last_close,buy_signal,sell_signal
0,GBP_JPY,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,161.56400,0,0
1,EUR_USD,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,1.06937,0,0
2,USD_JPY,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,134.16400,0,0
3,EUR_JPY,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,143.46800,0,0
4,AUD_JPY,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,92.27800,0,0
5,CAD_JPY,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,99.57200,0,0
6,CHF_JPY,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,145.18000,0,0
7,GBP_USD,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,1.20420,0,0
8,USD_CAD,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,1.34740,0,0
9,USD_CHF,2023-02-17 21:55:00+00:00,2023-02-17 16:55:00-05:00,0.92414,0,0


In [207]:
cur_time = datetime.datetime.now()
cur_time

datetime.datetime(2023, 2, 17, 17, 2, 53, 151183)

In [208]:
cur_time.year

2023

In [209]:
cur_time.month

2

In [210]:
cur_time.day

17

In [211]:
cur_time.hour

17

In [212]:
cur_time.minute

2

In [215]:
 %5

2

In [221]:
5 // 5

1

In [224]:
today = date.today()
dir_name = os.getcwd() + '/trading_folder/' + str(today)

isdir = os.path.isdir(dir_name) 
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name)   
dir_name

MAKING DIRECTORY: /Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/trading_folder/2023-02-17


'/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/trading_folder/2023-02-17'

In [ ]:
except FileNotFoundError:
    print("FileNotFoundError: running loop once to get file")


# Check if we are at an entry point

In [278]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd


import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 

#import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import numpy as np
import datetime
from scipy import stats

def convert_to_simple_df(instrument,first = True,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    #print(instrument,granularity,count)
    if first:
        params = {"count":count,
              "granularity": granularity
                 }
    else:
        params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }

    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')   
   # print('ACTUAL SHAPE:',df.shape,' MIN UTC:',df['utc_timestamp'].min(),' MAX UTC:',df['utc_timestamp'].max())
    return df
def add_feature_cols(df):
    
    def smma(df,period = 14):
        close_col = df.columns.get_loc('c')
        df['smma_' + str(period)] = df['c']
        smma_col = df.columns.get_loc('smma_' + str(period))
        arr = df.values

        #SMMA CALC:

        #SUM1=SUM (CLOSE, N)

        #SMMA1 = SUM1/ N

        #The second and subsequent moving averages are calculated according to this formula:

        #SMMA (i) = (SUM1 – SMMA1+CLOSE (i))/ N    

       # Where:

        #SUM1 – is the total sum of closing prices for N periods;
        #SMMA1 – is the smoothed moving average of the first bar;
        #SMMA (i) – is the smoothed moving average of the current bar (except the first one);
        #CLOSE (i) – is the current closing price;
        #N – is the smoothing period.    

        for i in range(period,arr.shape[0]):
            if i == period:
                sum1 = sum(arr[:i,close_col])

                smma1 = sum1 / period

                arr[i,smma_col] = smma1
            elif i == period + 1:
                arr[i,smma_col] = (smma1 * (period - 1) + arr[i,close_col]) / period

            else:

                prev_sum = arr[i - 1,smma_col] * period
                arr[i,smma_col] = (prev_sum - arr[i - 1,smma_col] + arr[i,close_col]) / period


        return pd.DataFrame(arr,columns = df.columns)    
    s = time.time()
    
    df['o'] = df['o'].astype(float)
    df['h'] = df['h'].astype(float)
    df['l'] = df['l'].astype(float)
    df['c'] = df['c'].astype(float)
    
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)

    df['delta_vector'] = 0

    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')

    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    e = time.time()
   # print('ADDING FEATURE COLS TIME:',round(e-s,3),'SECONDS')
    return df



def create_data(instrument,data_segments,count,pair,hours = 3,granularity = "M1"):

 #   print('CREATING DATASET...')
    start_ = time.time()
  #  print('PARAMS:',pair,granularity)

    total_df = convert_to_simple_df(instrument = instrument,first = True,granularity = granularity,count = count,start = datetime.datetime(2023,1,1,0,0,0),end = datetime.datetime(2023,1,1,0,0,0))
    cur = total_df['utc_timestamp'].iloc[0]
    df_lst = [total_df]
    for i in range(data_segments):
        df = convert_to_simple_df(instrument = instrument,first = False,granularity = granularity,count = count,start = cur - timedelta(hours = hours),end = cur)
        cur = cur - timedelta(hours = hours)
        df_lst.append(df)
       # total_df = pd.concat([total_df,df],axis = 0)
        #total_df = total_df.append(df)
    
    for item in enumerate(reversed(df_lst)):
        if item[0] == 0:
            total_df = item[1]
        else:
            total_df = pd.concat([total_df,item[1]],axis = 0)
    total_df = add_feature_cols(total_df)
   # print(total_df.shape)
#
    end_ = time.time()
   # print('TOTAL DATASET CONSTRUCTION TIME:',round(end_-start_,3),' SECONDS')
    return total_df



def signal_check(df,
                lookup_range = 1500,
                bullish_ma = True,
                bullish_candle = True,
                candle_size = .001,
                delta_filter = .001,
                delta_filter2 = .006,
                upper_wick = .0001,
                lower_wick = .0001,
                support = True                
                ):
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    ma1_col = df.columns.get_loc('smma_21')
    ma2_col = df.columns.get_loc('smma_50')
    ma3_col = df.columns.get_loc('smma_200')  
     
    
    
    i = df.shape[0] - 1
    #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range
    max_ = df.iloc[i - lookup_range : i,c_col].max()
    min_ = df.iloc[i - lookup_range : i,c_col].min()
    resistance_delta = max_ - df.iloc[i,c_col] #arr[i,c_col]
    support_delta = df.iloc[i,c_col] - min_ #arr[i,c_col] - min_ 
    pip_range = max_ - min_    
    if bullish_ma == True:
        if df.iloc[i,ma1_col] > df.iloc[i,ma2_col] and df.iloc[i,ma2_col] > df.iloc[i,ma3_col]:
            ma_indicator = 1
        else:
            ma_indicator = 0
    elif bullish_ma == False:
        if df.iloc[i,ma1_col] < df.iloc[i,ma2_col] and df.iloc[i,ma2_col] < df.iloc[i,ma3_col]:
            ma_indicator = 1
        else:
            ma_indicator = 0      
            
            
    #CANDLE BULL OR BEAR
    if df.iloc[i,c_col] >= df.iloc[i,o_col]:
        bull = True
    else:
        bull = False      

    #Within the delta range:
    if support == True:
        if support_delta >= delta_filter and support_delta <= delta_filter + delta_filter2:
            near_reversal_indicator = 1
        else:
            near_reversal_indicator = 0
    else:
        if resistance_delta >= delta_filter and resistance_delta <= delta_filter + delta_filter2:
            near_reversal_indicator = 1
        else:
            near_reversal_indicator = 0            

    #within support and resistance range:
    if delta_filter < 0:
        if df.iloc[i,c_col] >= min_ - abs(delta_filter) and df.iloc[i,c_col] <= max_ + abs(delta_filter):
            between_s_and_r = 1
        else:
            between_s_and_r = 0            
    else:
        if df.iloc[i,c_col] >= min_ and df.iloc[i,c_col] <= max_:
            between_s_and_r = 1
        else:
            between_s_and_r = 0

    if between_s_and_r and \
    near_reversal_indicator == 1 and \
    ma_indicator == 1 and \
    df.iloc[i,d_col] >= candle_size and \
    df.iloc[i,uw_col] >= upper_wick and \
    df.iloc[i,lw_col] >= lower_wick and \
    bull == bullish_candle:
        return 1
    else:
        return 0 
def check_all_pairs(all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0):
    lst = []
    for pair in all_pairs:

        df = create_data(instrument = pair,data_segments = data_segments,count = count,pair = pair,hours = hours,granularity = "M5")
        last_utc = df['utc_timestamp'].iloc[-1]
        last_est = df['est_timestamp'].iloc[-1]
        last_close = df['c'].iloc[-1]

        
        #print(df.shape)    
        buy_or_sell = 'buy'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    

        buy_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
        buy_or_sell = 'sell'
        params = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]    
        

        sell_signal = signal_check(df,
            lookup_range = params['lookup_range'].iloc[0],
            bullish_ma = params['bullish_ma'].iloc[0],
            bullish_candle = params['bullish_candle'].iloc[0],
            candle_size = params['candle_size'].iloc[0],
            delta_filter = params['delta_filter'].iloc[0],
            delta_filter2 = params['delta_filter2'].iloc[0],
            upper_wick = params['upper_wick'].iloc[0],
            lower_wick = params['lower_wick'].iloc[0],
            support = params['support'].iloc[0]                
            )
      #  print(buy_signal,sell_signal)
      #  if buy_signal == 1 and placed_buy_in_last_24_candles == 0:
       #     print('BUYING')
        #if sell_signal == 1 and placed_sell_in_last_24_candles == 0:
         #   print('SELLING')
        #if buy_signal + sell_signal == 0 or placed_buy_in_last_24_candles + placed_sell_in_last_24_candles != 0:
         #   print('NOT PLACING TRADE')
            
        lst.append([pair,str_time,last_utc,last_est,last_close,buy_signal,sell_signal])
        
        
    return pd.DataFrame(lst,columns = ['pair','sys_time_5_min','last_utc','last_est','last_close','buy_signal','sell_signal'])
            
   
    
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 

file = 'best_trade_params.csv'
param_df = pd.read_csv(file)
count = 5000
data_segments = 5
hours = 250

dir_name = os.getcwd() + '/trading_folder'

isdir = os.path.isdir(dir_name) 
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name) 
today = date.today()
dir_name = os.getcwd() + '/trading_folder/' + str(today)

isdir = os.path.isdir(dir_name) 
if isdir:
    pass
else:
    print('MAKING DIRECTORY:',dir_name)
    os.mkdir(dir_name)     
    
    
#start = datetime.datetime(2023,1,1,0,0,0)
#end = datetime.datetime(2023,7,31,0,0,0)
s = time.time()

trade_check_path = dir_name + '/trade_check_data.csv'
cur_time = datetime.datetime.now()
str_time = str(cur_time.year) + '_' + str(cur_time.month) + '_' + str(cur_time.day) + '_' + str(cur_time.hour) + '_' + str((cur_time.minute // 5) * 5)
print('CUR TIME:',str_time)
try:
    trade_check_df = pd.read_csv(trade_check_path)

    if str_time in list(trade_check_df['sys_time_5_min']):
        print(str_time,'ALREADY EXISTS, PASSING')
    else:
        print('DOES NOT EXIST, RUNNING TRADE CHECKER')
        check_df = check_all_pairs(all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 
        
        trade_check_df = pd.concat([trade_check_df,check_df],axis = 0)
        trade_check_df.to_csv(trade_check_path,index = False)
        
except FileNotFoundError:
    print("FileNotFoundError: running loop once to get file")
    check_df = check_all_pairs(all_pairs,str_time,placed_buy_in_last_24_candles = 0,placed_sell_in_last_24_candles = 0) 
    check_df.to_csv(trade_check_path,index = False)

e = time.time()
print((e-s),'TOTAL SECONDS')



CUR TIME: 2023_2_17_18_10
DOES NOT EXIST, RUNNING TRADE CHECKER
19.114770889282227 TOTAL SECONDS


BUY LEN: 0 SELL LEN: 0


# Enter the trade

In [281]:
def get_account_balance():
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    bal = acc_details['account']['balance']
    margin_available = acc_details['account']['marginAvailable']
    print('CURRENT BALANCE:',round(float(bal)),'CURRENT MARGIN AVAILABLE',round(float(margin_available)))
    return min(float(bal),float(margin_available))

def units_calc(trading_account,max_risk,sl):
    units=round((max_risk * trading_account / sl) * 10000)
    print('MAX RISK:',max_risk,'SL:',sl,'LOTS:',round(units / 100000,2),'UNITS:',units)
    return units
def create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS):
    def check_price(pair):
        api = oandapyV20.API(access_token=access_token)
        bid_ask=[]
        params={"instruments":pair}

        r = pricing.PricingInfo(accountID=accountID, params=params)
        a = api.request(r)

        bid=float(a['prices'][0]['bids'][0]['price'])
        ask=float(a['prices'][0]['asks'][0]['price'])
        avg=(bid+ask)/2

        bid_ask.append(bid)
        bid_ask.append(ask)
        bid_ask.append(avg)



        return bid_ask
    
    if 'JPY' in instrument:
        multiplier = .01
    else:
        multiplier = .0001
    if units > 0:
        price = min(check_price(instrument))
        TAKE_PROFIT = round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price - (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (STOP_LOSS_NUM_PIPS * multiplier),5)
    else:
        price = max(check_price(instrument))
        TAKE_PROFIT = round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price + (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (STOP_LOSS_NUM_PIPS * multiplier),5)
        
        
    print('CREATING MARKET ORDER:',instrument,'|LOTS:',units / 100000,'|UNITS:',units,'|TP NUM PIPS:',TAKE_PROFIT_NUM_PIPS,
          '|SL NUM PIPS:',STOP_LOSS_NUM_PIPS,'|CUR PRICE:',price,'|STOP LOSS:',STOP_LOSS,'|TAKE PROFIT:',TAKE_PROFIT)
    print()
    
    api = oandapyV20.API(access_token=access_token)

    mktOrder = MarketOrderRequest(
        instrument = instrument,
        units = units,
        takeProfitOnFill=TakeProfitDetails(price=TAKE_PROFIT).data,
        stopLossOnFill=StopLossDetails(price=STOP_LOSS).data)

    # create the OrderCreate request
    r = orders.OrderCreate(accountID, data=mktOrder.data)
    try:
        # create the OrderCreate request
        rv = api.request(r)
    except oandapyV20.exceptions.V20Error as err:
        print(r.status_code, err)
    else:
        print(json.dumps(rv, indent=2))
        
    return rv
def enter_trade(param_df,pair,buy_or_sell):
    # get stop loss and take profit
    sl = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['sl'].iloc[0]
    tp = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['tp'].iloc[0]


    # get balance and estimated units to trade
    if 'JPY' in pair:
        TAKE_PROFIT_NUM_PIPS = tp * 100
        STOP_LOSS_NUM_PIPS = sl * 100
    else:
        TAKE_PROFIT_NUM_PIPS = tp * 100 * 100
        STOP_LOSS_NUM_PIPS = sl * 100 * 100

    balance = get_account_balance()
    units = units_calc(trading_account = balance,max_risk = .01,sl = int_sl)

    instrument =  pair
    if buy_or_sell != 'buy':
        units = units * -1
        
    order_dict = create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS)
    # save order dict.  order_dict
    
    
pair = 'GBP_JPY'

buy_or_sell = 'buy'

buys = list(check_df[check_df['buy_signal'] == 1]['pair'])
sells = list(check_df[check_df['buy_signal'] == 1]['pair'])
print('BUY LEN:',len(buys),'SELL LEN:',len(sells))
for pair in buys:
    print(pair)
    enter_trade(param_df,pair,buy_or_sell = 'buy')
for pair in sells:
    print(pair)
    enter_trade(param_df,pair,buy_or_sell = 'sell')


BUY LEN: 0 SELL LEN: 0


# TO DO
## Save order dict
## check if trade has been placed on the pair in the last 24 candles


In [276]:
type(rv)

dict

In [261]:
sl * 100

5.0

In [262]:
pair = 'EUR_USD'

buy_or_sell = 'buy'
# get stop loss and take profit
sl = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['sl'].iloc[0]
tp = param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]['tp'].iloc[0]
sl

0.0005

In [263]:
sl * 100 * 100

5.0

In [259]:
def get_account_balance():
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    bal = acc_details['account']['balance']
    margin_available = acc_details['account']['marginAvailable']
    print(bal,margin_available)
    return min(float(bal),float(margin_available))

def units_calc(trading_account,max_risk,sl):
    units=(max_risk * trading_account / sl) * 10000
    print('MAX RISK:',max_risk,'SL:',sl,'LOTS:',round(units / 100000,2),'UNITS:',units)
    return units

balance = get_account_balance()
units = units_calc(trading_account = balance,max_risk = .01,sl = 5)
units

98372.2175 98372.2175
MAX RISK: 0.01 SL: 5 LOTS: 19.67 UNITS: 1967444.35


1967444.35

In [288]:
def close_trade(trade_id):
    print('CLOSING TRADE:',trade_id)
    api = oandapyV20.API(access_token=access_token)
    r = trades.TradeClose(accountID = accountID,tradeID = trade_id)
    t = api.request(r)
    print(t)
api = oandapyV20.API(access_token=access_token)
r = positions.OpenPositions(accountID)
pos = api.request(r)
close_trade(pos['positions'][0][long_or_short]['tradeIDs'][0])      


CLOSING TRADE: 211
{'orderCreateTransaction': {'id': '214', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '214', 'requestID': '25047850733947167', 'time': '2023-02-20T15:08:05.251697727Z', 'type': 'MARKET_ORDER', 'instrument': 'GBP_JPY', 'units': '-1967444', 'timeInForce': 'FOK', 'positionFill': 'REDUCE_ONLY', 'reason': 'TRADE_CLOSE', 'tradeClose': {'units': 'ALL', 'tradeID': '211'}}, 'orderFillTransaction': {'id': '215', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '214', 'requestID': '25047850733947167', 'time': '2023-02-20T15:08:05.251697727Z', 'type': 'ORDER_FILL', 'orderID': '214', 'instrument': 'GBP_JPY', 'units': '-1967444', 'requestedUnits': '-1967444', 'price': '161.255', 'pl': '-250.8800', 'quotePL': '-33446.548', 'financing': '0.0000', 'baseFinancing': '0.00000000000000', 'commission': '0.0000', 'accountBalance': '98121.3375', 'gainQuoteHomeConversionFactor': '0.007426287595', 'lossQuoteHomeConversionFactor': '0.007500923651', 'gu

# Payload
## 100000 units = 1 lot

In [152]:
def close_trade(trade_id):
    print('CLOSING TRADE:',trade_id)
    api = oandapyV20.API(access_token=access_token)
    r = trades.TradeClose(accountID = accountID,tradeID = trade_id)
    t = api.request(r)
    print(t)
    
    
pair = 'USD_JPY'
pair = 'EUR_USD'
units = 4000000
print('TRADING',units / 100000,'LOTS AND',units,'UNITS')
long_or_short = 'long'
if long_or_short == 'long':
    units = abs(units)
else:
    units = -1 * abs(units)
data = {
  "order": {
    "units": str(units),
    "instrument": pair,
    "timeInForce": "FOK",
    "type": "MARKET",
    "positionFill": "DEFAULT"
  }
}

r = orders.OrderCreate(accountID, data=data)
client.request(r)
print(r.response)
api = oandapyV20.API(access_token=access_token)
r = positions.OpenPositions(accountID)
pos = api.request(r)
close_trade(pos['positions'][0][long_or_short]['tradeIDs'][0])  



TRADING 40.0 LOTS AND 4000000 UNITS
{'orderCreateTransaction': {'id': '152', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '152', 'requestID': '43060536759671319', 'time': '2023-02-15T21:43:17.147655607Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '4000000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '153', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '152', 'requestID': '43060536759671319', 'time': '2023-02-15T21:43:17.147655607Z', 'type': 'ORDER_FILL', 'orderID': '152', 'instrument': 'EUR_USD', 'units': '4000000', 'requestedUnits': '4000000', 'price': '1.06897', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.0000', 'accountBalance': '98932.1982', 'gainQuoteHomeConversionFactor': '1', 'lossQuoteHomeConversionFactor': '1', 'guaranteedExecutionFee': '0.0000', 'quoteGuaranteedExecutionFee': '0', 'halfSpreadCost': '280.

In [164]:
stop_loss_pips = 1
instrument = 'EUR_USD'

# Connect to the OANDA API
client = oandapyV20.API(access_token=access_token)

# Get the current account details
account_info = accounts.AccountSummary(accountID=accountID)
account_data = client.request(account_info)

# Get the current pricing for the instrument
pricing_info = pricing.PricingInfo(accountID=accountID, params={"instruments": instrument})
pricing_data = client.request(pricing_info)
price = float(pricing_data["prices"][0]["bids"][0]["price"])

# Calculate the maximum number of units for the trade
margin_rate = float(account_data["account"]["marginRate"])
margin_available = float(account_data["account"]["marginAvailable"])
max_trade_size = margin_available / (margin_rate * price * stop_loss_pips)

# Print the maximum number of units for the trade
print("Maximum Trade Size: {:.2f}".format(max_trade_size))

Maximum Trade Size: 4602337.29


In [150]:
def calculate_trade_size(access_token, accountID, instrument, account_balance, stop_loss, risk_percentage):
    # Connect to the OANDA API
    client = oandapyV20.API(access_token=access_token)

    # Get the current account details
    account_info = accounts.AccountSummary(accountID=accountID)
    account_data = client.request(account_info)

    # Get the current pricing for the instrument
    pricing_info = pricing.PricingInfo(accountID=accountID, params={"instruments": instrument})
    pricing_data = client.request(pricing_info)
    price = float(pricing_data["prices"][0]["bids"][0]["price"])

    # Calculate the maximum risk per trade
    max_risk = account_balance * (risk_percentage / 100)

    # Calculate the trade size in units
    margin_rate = float(account_data["account"]["marginRate"])
    print(margin_rate)
    trade_size = (max_risk / (price - stop_loss)) / (margin_rate * price)

    return trade_size

# Example usage of the function

instrument = "EUR_USD"
#instrument = "USD_JPY"
account_balance = 100000
stop_loss = .001
#stop_loss = .1
risk_percentage = 1
trade_size = calculate_trade_size(access_token, accountID, instrument, account_balance, stop_loss, risk_percentage)
print("Trade Size: {:.2f}".format(trade_size))

0.02
Trade Size: 43809.41


In [194]:
def get_account_balance():
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    bal = acc_details['account']['balance']
    margin_available = acc_details['account']['marginAvailable']
    print(bal,margin_available)
    return min(float(bal),float(margin_available))

def units_calc(trading_account,max_risk,sl):
    units=(max_risk * trading_account / sl) * 10000
    print('MAX RISK:',max_risk,'SL:',sl,'LOTS:',units / 100000,'UNITS:',units)
    return units


balance = get_account_balance()
units = units_calc(trading_account = balance,max_risk = .01,sl = 5)
units

98372.2175 98372.2175
MAX RISK: 0.01 SL: 5 LOTS: 19.674443500000002 UNITS: 1967444.35


1967444.35

In [138]:
account_balance = 100000
risk = .01
risk_amount = account_balance * risk
risk_amount

1000.0

In [165]:
100 * 20 / 6

333.3333333333333

In [151]:
def units_calc(trading_account,max_risk,sl):
    units=(max_risk * trading_account / sl) * 10000
    
    return units
units = units_calc(trading_account = 100000,max_risk = .01,sl = 10)
print(units / 100000,units)

10.0 1000000.0


In [127]:
margin_rate

0.02

In [128]:
margin_available

98932.1982

In [129]:
price

134.116

In [122]:
margin_rate

'0.02'

In [123]:
price

134.11

In [124]:
stop_loss_pips

10

In [125]:
account_data["account"]["marginAvailable"] 

'98932.1982'

In [109]:
api = oandapyV20.API(access_token=access_token)
r = positions.OpenPositions(accountID)
pos=api.request(r)
pos['positions'][0]['short']['tradeIDs'][0]

'123'

# Placing market order 
## from https://pypi.org/project/oandapyV20/

In [187]:
import json

from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

import oandapyV20.endpoints.orders as orders
import oandapyV20



def create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS):
    def check_price(pair):
        api = oandapyV20.API(access_token=access_token)
        bid_ask=[]
        params={"instruments":pair}

        r = pricing.PricingInfo(accountID=accountID, params=params)
        a = api.request(r)

        bid=float(a['prices'][0]['bids'][0]['price'])
        ask=float(a['prices'][0]['asks'][0]['price'])
        avg=(bid+ask)/2

        bid_ask.append(bid)
        bid_ask.append(ask)
        bid_ask.append(avg)



        return bid_ask
    
    if 'JPY' in instrument:
        multiplier = .01
    else:
        multiplier = .0001
    if units > 0:
        price = min(check_price(instrument))
        TAKE_PROFIT = round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price - (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (STOP_LOSS_NUM_PIPS * multiplier),5)
    else:
        price = max(check_price(instrument))
        TAKE_PROFIT = round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price - (TAKE_PROFIT_NUM_PIPS * multiplier),5)
        STOP_LOSS = round(price + (STOP_LOSS_NUM_PIPS * multiplier),3) if 'JPY' in instrument else round(price + (STOP_LOSS_NUM_PIPS * multiplier),5)
        
        
    print('CREATING MARKET ORDER:',instrument,'|LOTS:',units / 100000,'|UNITS:',units,'|TP NUM PIPS:',TAKE_PROFIT_NUM_PIPS,
          '|SL NUM PIPS:',STOP_LOSS_NUM_PIPS,'|CUR PRICE:',price,'|STOP LOSS:',STOP_LOSS,'|TAKE PROFIT:',TAKE_PROFIT)
    
    api = oandapyV20.API(access_token=access_token)

    mktOrder = MarketOrderRequest(
        instrument = instrument,
        units = units,
        takeProfitOnFill=TakeProfitDetails(price=TAKE_PROFIT).data,
        stopLossOnFill=StopLossDetails(price=STOP_LOSS).data)

    # create the OrderCreate request
    r = orders.OrderCreate(accountID, data=mktOrder.data)
    try:
        # create the OrderCreate request
        rv = api.request(r)
    except oandapyV20.exceptions.V20Error as err:
        print(r.status_code, err)
    else:
        print(json.dumps(rv, indent=2))
        
instrument = 'USD_JPY'
units = 1000
TAKE_PROFIT_NUM_PIPS = 30
STOP_LOSS_NUM_PIPS = 10
create_market_order(instrument,units,TAKE_PROFIT_NUM_PIPS,STOP_LOSS_NUM_PIPS)



CREATING MARKET ORDER: USD_JPY |LOTS: 0.01 |UNITS: 1000 |TP NUM PIPS: 30 |SL NUM PIPS: 10 |CUR PRICE: 133.805 |STOP LOSS: 133.705 |TAKE PROFIT: 134.105
{
  "orderCreateTransaction": {
    "id": "188",
    "accountID": "101-001-20062555-002",
    "userID": 20062555,
    "batchID": "188",
    "requestID": "43060574392273463",
    "time": "2023-02-16T00:12:49.933115750Z",
    "type": "MARKET_ORDER",
    "instrument": "USD_JPY",
    "units": "1000",
    "timeInForce": "FOK",
    "positionFill": "DEFAULT",
    "takeProfitOnFill": {
      "price": "134.105",
      "timeInForce": "GTC"
    },
    "stopLossOnFill": {
      "price": "133.705",
      "timeInForce": "GTC",
      "triggerMode": "TOP_OF_BOOK"
    },
    "reason": "CLIENT_ORDER"
  },
  "orderFillTransaction": {
    "id": "189",
    "accountID": "101-001-20062555-002",
    "userID": 20062555,
    "batchID": "188",
    "requestID": "43060574392273463",
    "time": "2023-02-16T00:12:49.933115750Z",
    "type": "ORDER_FILL",
    "orderI

In [176]:
round(1.36927,3)

1.369

In [189]:
trade_request = trades.OpenTrades(accountID=accountID)
trade_response = api.request(trade_request)

# Close out all open trades
for trade in trade_response['trades']:
    print(trade)
    trade_id = trade['id']
    close_trade_request = trades.TradeClose(accountID=accountID, tradeID=trade_id)
    api.request(close_trade_request)

print('All open trades have been closed')

All open trades have been closed


In [169]:
10 * .01

0.1

In [92]:
def close_trade(trade_id):
    print('CLOSING TRADE:',trade_id)
    api = oandapyV20.API(access_token=access_token)
    r = trades.TradeClose(accountID = accountID,tradeID = trade_id)
    t = api.request(r)
    print(t)
close_trade(trade_id = '7')



CLOSING TRADE: 7
{'orderCreateTransaction': {'id': '116', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '116', 'requestID': '79089320154757380', 'time': '2023-02-15T20:49:09.768623917Z', 'type': 'MARKET_ORDER', 'instrument': 'USD_JPY', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'REDUCE_ONLY', 'reason': 'TRADE_CLOSE', 'tradeClose': {'units': 'ALL', 'tradeID': '7'}}, 'orderFillTransaction': {'id': '117', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '116', 'requestID': '79089320154757380', 'time': '2023-02-15T20:49:09.768623917Z', 'type': 'ORDER_FILL', 'orderID': '116', 'instrument': 'USD_JPY', 'units': '1000', 'requestedUnits': '1000', 'price': '134.153', 'pl': '105.1561', 'quotePL': '14177.000', 'financing': '0.0000', 'baseFinancing': '0.00000000000000', 'commission': '0.0000', 'accountBalance': '100094.9593', 'gainQuoteHomeConversionFactor': '0.007417374577', 'lossQuoteHomeConversionFactor': '0.007491921056', 'guaranteedExecution

In [156]:
api = oandapyV20.API(access_token=access_token)
r = accounts.AccountDetails(accountID)
acc_details=api.request(r)
bal = acc_details['account']['balance']
margin_available = acc_details['account']['marginAvailable']
print(bal,margin_available)

98372.1982 98372.1982


In [132]:
api = oandapyV20.API(access_token=access_token)
r = accounts.AccountSummary(accountID)
summary=api.request(r)



In [68]:
def account_value():
        
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    
    balance=float(acc_details['account']['balance'])
    margin_available=float(acc_details['account']['marginAvailable'])
    open_trades=float(acc_details['account']['openTradeCount'])
    
    return balance,margin_available,open_trades
balance,margin_available,open_trades = account_value()
balance

99989.8032

In [167]:
params={"instruments":pair}

r = pricing.PricingInfo(accountID=accountID, params=params)
a=api.request(r)
a

{'time': '2023-02-15T23:25:57.126495186Z',
 'prices': [{'type': 'PRICE',
   'time': '2023-02-15T23:25:56.179369337Z',
   'bids': [{'price': '1.06864', 'liquidity': 10000000}],
   'asks': [{'price': '1.06881', 'liquidity': 10000000}],
   'closeoutBid': '1.06864',
   'closeoutAsk': '1.06881',
   'status': 'tradeable',
   'tradeable': True,
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'EUR_USD'}]}

In [70]:
open_trades

1.0

In [73]:
from oandapyV20.contrib.requests import MarketOrderRequest

In [168]:
def check_price(pair):
    bid_ask=[]
    params={"instruments":pair}
            
    r = pricing.PricingInfo(accountID=accountID, params=params)
    a=api.request(r)
    
    bid=float(a['prices'][0]['bids'][0]['price'])
    ask=float(a['prices'][0]['asks'][0]['price'])
    avg=(bid+ask)/2
    
    bid_ask.append(bid)
    bid_ask.append(ask)
    bid_ask.append(avg)
    
    
    
    return bid_ask
bid_ask = check_price(pair)
bid_ask

[1.06862, 1.0688, 1.0687099999999998]

In [ ]:
def account_value():
        
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    
    balance=float(acc_details['account']['balance'])
    margin_available=float(acc_details['account']['marginAvailable'])
    open_trades=float(acc_details['account']['openTradeCount'])
    
    return balance,margin_available,open_trades


def units_calc(trading_account,max_risk,sl):
    units=(max_risk*trading_account/sl)*10000
    
    return units

def stoploss_calc(pair,wick,sl,rev):
    sl_price=0
    if 'JPY' in pair:
        sl=sl*.01
        if rev==1:
            sl_price=wick-sl
        elif rev==2:
            sl_price=wick+sl
    else:
        sl=sl*.0001
        if rev==1:
            sl_price=wick-sl
        elif rev==2:
            sl_price=wick+sl    
    return sl_price
    
def takeprofit_calc(pair,current_price,tp,rev):
    tp_price=0
    if 'JPY' in pair:
        tp=tp*.01
        if rev==1:
            tp_price=current_price+tp
        elif rev==2:
            tp_price=current_price-tp
    else:
        tp=tp*.0001
        if rev==1:
            tp_price=current_price+tp
        elif rev==2:
            tp_price=current_price-tp  
    return tp_price


def jpy_trade(sl,tp,pair,current_price,wick,rev):
    bal,marg,trades=account_value()
    units=str(int(units_calc(bal,.01,sl)))
    stoploss=str(stoploss_calc(pair,wick,sl,rev))
    takeprofit=str(takeprofit_calc(pair,current_price,tp,rev))
    
    order={"order": 
        {
        "type": "MARKET",
        "units": units,
        "instrument": pair,
        "timeInForce": "FOK",
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": takeprofit
        },
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": stoploss}
        }
        }
    r=orders.OrderCreate(accountID,data=order)
    api.request(r)
    

    
def majors_trade(sl,tp,pair,current_price,wick,rev):
    bal,marg,trades=account_value()
    units=str(int(units_calc(bal,.01,sl)))
    stoploss=str(stoploss_calc(pair,wick,sl,rev))
    takeprofit=str(takeprofit_calc(pair,current_price,tp,rev))
    
    order={"order": 
        {
        "type": "MARKET",
        "units": units,
        "instrument": pair,
        "timeInForce": "FOK",
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": takeprofit
        },
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": stoploss}
        }
        }
    r=orders.OrderCreate(accountID,data=order)
    api.request(r)    
    
    
    

def cross_trade(sl,tp,pair,current_price,wick,rev):
    bal,marg,trades=account_value()
    units=str(int(units_calc(bal,.01,sl)))
    stoploss=str(stoploss_calc(pair,wick,sl,rev))
    takeprofit=str(takeprofit_calc(pair,current_price,tp,rev))
    
    order={"order": 
        {
        "type": "MARKET",
        "units": units,
        "instrument": pair,
        "timeInForce": "FOK",
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": takeprofit
        },
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": stoploss}
        }
        }
    r=orders.OrderCreate(accountID,data=order)
    api.request(r)

In [67]:
file = 'best_trade_params.csv'
param_df = pd.read_csv(file)
print(param_df.shape)
param_df.head()

(30, 17)


,instrument,buy_or_sell,ending_val,sl,tp,lookup_range,bullish_ma,bullish_candle,support,candle_size,delta_filter,delta_filter2,upper_wick,lower_wick,RR,win_rate,actual_number_of_trades
0,AUD_CAD,buy,92757.5,0.00125,0.0090,500,True,False,True,0.00050,0.0020,0.0005,0.0000,0.0000,7.2,0.1967,122
1,AUD_CAD,sell,206865.5,0.00125,0.0090,1000,False,True,False,0.00005,0.0005,0.0015,0.0000,0.0000,7.2,0.2128,94
2,AUD_CHF,buy,52802.5,0.00050,0.0015,750,True,False,False,0.00150,0.0005,0.0070,0.0000,0.0000,3.0,0.3587,92
3,AUD_CHF,sell,143577.0,0.00125,0.0090,500,False,True,True,0.00050,0.0015,0.0070,0.0000,0.0004,7.2,0.2093,86
4,AUD_NZD,buy,245188.0,0.00075,0.0090,500,True,False,True,0.00075,0.0005,0.0070,0.0002,0.0000,12.0,0.1429,91


In [50]:
buy_or_sell = 'buy'
param_df[(param_df['instrument'] == pair) & (param_df['buy_or_sell'] == buy_or_sell)]

,instrument,buy_or_sell,ending_val,sl,tp,lookup_range,bullish_ma,bullish_candle,support,candle_size,delta_filter,delta_filter2,upper_wick,lower_wick
0,AUD_CAD,buy,92757.5,0.00125,0.009,500,True,False,True,0.0005,0.002,0.0005,0.0,0.0


In [ ]:
[150,250,400,500,750,1000]

In [15]:
df.head()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,smma_200,upper_wick,lower_wick
0,2022-11-28T15:45:00.000000000Z,445,0.89948,0.90029,0.89914,0.89998,2022-11-28 15:45:00+00:00,2022-11-28 10:45:00-05:00,0.89998,0.89998,0.89998,0.00031,0.00034
1,2022-11-28T15:50:00.000000000Z,476,0.89994,0.90003,0.89895,0.89912,2022-11-28 15:50:00+00:00,2022-11-28 10:50:00-05:00,0.89912,0.89912,0.89912,0.00009,0.00017
2,2022-11-28T15:55:00.000000000Z,745,0.89912,0.89912,0.89854,0.89882,2022-11-28 15:55:00+00:00,2022-11-28 10:55:00-05:00,0.89882,0.89882,0.89882,0.0,0.00028
3,2022-11-28T16:00:00.000000000Z,517,0.89882,0.89902,0.89848,0.89876,2022-11-28 16:00:00+00:00,2022-11-28 11:00:00-05:00,0.89876,0.89876,0.89876,0.0002,0.00028
4,2022-11-28T16:05:00.000000000Z,494,0.89872,0.8989,0.8986,0.8989,2022-11-28 16:05:00+00:00,2022-11-28 11:05:00-05:00,0.8989,0.8989,0.8989,0.0,0.00012


In [16]:
df.tail()

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,smma_200,upper_wick,lower_wick
15645,2023-02-14T02:05:00.000000000Z,368,0.92852,0.929,0.92846,0.92886,2023-02-14 02:05:00+00:00,2023-02-13 21:05:00-05:00,0.928894,0.928722,0.927695,0.00014,0.00006
15646,2023-02-14T02:10:00.000000000Z,191,0.92886,0.92898,0.9287,0.92885,2023-02-14 02:10:00+00:00,2023-02-13 21:10:00-05:00,0.928892,0.928724,0.927701,0.00012,0.00015
15647,2023-02-14T02:15:00.000000000Z,184,0.92886,0.92886,0.92844,0.92858,2023-02-14 02:15:00+00:00,2023-02-13 21:15:00-05:00,0.928877,0.928722,0.927705,0.0,0.00014
15648,2023-02-14T02:20:00.000000000Z,205,0.9285,0.92851,0.92804,0.92825,2023-02-14 02:20:00+00:00,2023-02-13 21:20:00-05:00,0.928847,0.928712,0.927708,0.00001,0.00021
15649,2023-02-14T02:25:00.000000000Z,162,0.92822,0.9283,0.92776,0.92776,2023-02-14 02:25:00+00:00,2023-02-13 21:25:00-05:00,0.928795,0.928693,0.927708,0.00008,0.0


In [18]:
df.iloc[df.shape[0] - 1,:]

time             2023-02-14T02:25:00.000000000Z
volume                                      162
o                                       0.92822
h                                        0.9283
l                                       0.92776
c                                       0.92776
utc_timestamp         2023-02-14 02:25:00+00:00
est_timestamp         2023-02-13 21:25:00-05:00
smma_21                                0.928795
smma_50                                0.928693
smma_200                               0.927708
upper_wick                              0.00008
lower_wick                                  0.0
Name: 15649, dtype: object

In [36]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.definitions.primitives as primitives
import pandas as pd
from dateutil.relativedelta import relativedelta
import pandas as pd


import os
import io
from io import StringIO 
import boto3
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time 

#import datedelta
import calendar
import  csv
import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import numpy as np
import datetime
from scipy import stats

def convert_to_simple_df(instrument,first = True,granularity = "H1",count = 5000,start = date(2020,1,1),end = date(2020,1,10)):
    print()
    print()
    print()
    print(instrument,granularity,count)
    print('START:',start,' END:',end)
    if first:
        params = {"count":count,
              "granularity": granularity
                 }
    else:
        params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:

        lst.append([candle['time'],
                    candle['volume'],
                    float(candle['mid']['o']),
                    float(candle['mid']['h']),
                    float(candle['mid']['l']),
                    float(candle['mid']['c'])
                   ])
        
 
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c'])
    
    df['utc_timestamp'] = pd.to_datetime(df['time'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
    df['est_timestamp'] = df['utc_timestamp'].dt.tz_convert('US/Eastern')   
    print('ACTUAL SHAPE:',df.shape,' MIN UTC:',df['utc_timestamp'].min(),' MAX UTC:',df['utc_timestamp'].max())
    return df
def add_feature_cols(df):
    
    def smma(df,period = 14):
        print('GETTING SMMA INDICATOR FOR:',period)
        close_col = df.columns.get_loc('c')
        df['smma_' + str(period)] = df['c']
        smma_col = df.columns.get_loc('smma_' + str(period))
        arr = df.values

        #SMMA CALC:

        #SUM1=SUM (CLOSE, N)

        #SMMA1 = SUM1/ N

        #The second and subsequent moving averages are calculated according to this formula:

        #SMMA (i) = (SUM1 – SMMA1+CLOSE (i))/ N    

       # Where:

        #SUM1 – is the total sum of closing prices for N periods;
        #SMMA1 – is the smoothed moving average of the first bar;
        #SMMA (i) – is the smoothed moving average of the current bar (except the first one);
        #CLOSE (i) – is the current closing price;
        #N – is the smoothing period.    

        for i in range(period,arr.shape[0]):
            if i == period:
                sum1 = sum(arr[:i,close_col])

                smma1 = sum1 / period

                arr[i,smma_col] = smma1
            elif i == period + 1:
                arr[i,smma_col] = (smma1 * (period - 1) + arr[i,close_col]) / period

            else:

                prev_sum = arr[i - 1,smma_col] * period
                arr[i,smma_col] = (prev_sum - arr[i - 1,smma_col] + arr[i,close_col]) / period


        return pd.DataFrame(arr,columns = df.columns)    
    s = time.time()
    
    df['o'] = df['o'].astype(float)
    df['h'] = df['h'].astype(float)
    df['l'] = df['l'].astype(float)
    df['c'] = df['c'].astype(float)
    
    df = smma(df,period = 21)
    df = smma(df,period = 50)
    df = smma(df,period = 200)

    df['delta_vector'] = 0

    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')

    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    e = time.time()
    print('ADDING FEATURE COLS TIME:',round(e-s,3),'SECONDS')
    return df



def create_data(instrument,data_segments,count,pair,hours = 3,granularity = "M1"):

    print('CREATING DATASET...')
    start_ = time.time()
    print('PARAMS:',pair,granularity)

    total_df = convert_to_simple_df(instrument = instrument,first = True,granularity = granularity,count = count,start = datetime.datetime(2023,1,1,0,0,0),end = datetime.datetime(2023,1,1,0,0,0))
    cur = total_df['utc_timestamp'].iloc[0]
    df_lst = [total_df]
    for i in range(data_segments):
        df = convert_to_simple_df(instrument = instrument,first = False,granularity = granularity,count = count,start = cur - timedelta(hours = hours),end = cur)
        cur = cur - timedelta(hours = hours)
        df_lst.append(df)
       # total_df = pd.concat([total_df,df],axis = 0)
        #total_df = total_df.append(df)
    
    for item in enumerate(reversed(df_lst)):
        if item[0] == 0:
            total_df = item[1]
        else:
            total_df = pd.concat([total_df,item[1]],axis = 0)
    total_df = add_feature_cols(total_df)
    print(total_df.shape)

    end_ = time.time()
    print('TOTAL DATASET CONSTRUCTION TIME:',round(end_-start_,3),' SECONDS')
    return total_df




accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)


all_pairs=['GBP_JPY','EUR_USD','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF'] 


#start = datetime.datetime(2023,1,1,0,0,0)
#end = datetime.datetime(2023,7,31,0,0,0)
#for pair in all_pairs:
pair = 'AUD_CAD'
count = 5000
data_segments = 5
hours = 250


df = create_data(instrument = pair,data_segments = data_segments,count = count,pair = pair,hours = hours,granularity = "M5")

print(df.shape)
df.head()



CREATING DATASET...
PARAMS: AUD_CAD M5



AUD_CAD M5 5000
START: 2023-01-01 00:00:00  END: 2023-01-01 00:00:00
PARAMS {'count': 5000, 'granularity': 'M5'}
AUD_CAD <class 'str'>
ACTUAL SHAPE: (5000, 8)  MIN UTC: 2023-01-20 14:00:00+00:00  MAX UTC: 2023-02-14 22:40:00+00:00



AUD_CAD M5 5000
START: 2023-01-10 04:00:00+00:00  END: 2023-01-20 14:00:00+00:00
PARAMS {'granularity': 'M5', 'from': '2023-01-10T04:00:00+00:00', 'to': '2023-01-20T14:00:00+00:00'}
AUD_CAD <class 'str'>
ACTUAL SHAPE: (2423, 8)  MIN UTC: 2023-01-10 04:00:00+00:00  MAX UTC: 2023-01-20 13:55:00+00:00



AUD_CAD M5 5000
START: 2022-12-30 18:00:00+00:00  END: 2023-01-10 04:00:00+00:00
PARAMS {'granularity': 'M5', 'from': '2022-12-30T18:00:00+00:00', 'to': '2023-01-10T04:00:00+00:00'}
AUD_CAD <class 'str'>
ACTUAL SHAPE: (1820, 8)  MIN UTC: 2022-12-30 18:00:00+00:00  MAX UTC: 2023-01-10 03:55:00+00:00



AUD_CAD M5 5000
START: 2022-12-20 08:00:00+00:00  END: 2022-12-30 18:00:00+00:00
PARAMS {'granularity': 'M5', 'from': 

,time,volume,o,h,l,c,utc_timestamp,est_timestamp,smma_21,smma_50,smma_200,delta_vector,upper_wick,lower_wick,delta
0,2022-11-29T12:00:00.000000000Z,400,0.90684,0.90702,0.9065,0.90678,2022-11-29 12:00:00+00:00,2022-11-29 07:00:00-05:00,0.90678,0.90678,0.90678,-0.00006,0.00018,0.00028,0.00006
1,2022-11-29T12:05:00.000000000Z,373,0.90674,0.90694,0.90657,0.90686,2022-11-29 12:05:00+00:00,2022-11-29 07:05:00-05:00,0.90686,0.90686,0.90686,0.00012,0.00008,0.00017,0.00012
2,2022-11-29T12:10:00.000000000Z,277,0.9069,0.90694,0.9065,0.90689,2022-11-29 12:10:00+00:00,2022-11-29 07:10:00-05:00,0.90689,0.90689,0.90689,-0.00001,0.00004,0.00039,0.00001
3,2022-11-29T12:15:00.000000000Z,276,0.90692,0.90702,0.9066,0.90668,2022-11-29 12:15:00+00:00,2022-11-29 07:15:00-05:00,0.90668,0.90668,0.90668,-0.00024,0.0001,0.00008,0.00024
4,2022-11-29T12:20:00.000000000Z,337,0.90667,0.90717,0.90662,0.90708,2022-11-29 12:20:00+00:00,2022-11-29 07:20:00-05:00,0.90708,0.90708,0.90708,0.00041,0.00009,0.00005,0.00041


In [65]:
best_trade_df.columns

Index(['instrument', 'timeframe', 'buy_or_sell', 'trade_strategy',
       'ending_val', 'starting_val', 'sl', 'tp', 'RR', 'win_rate',
       'num_trades', 'risk_size', 'actual_number_of_trades', 'lookup_range',
       'bullish_ma', 'bullish_candle', 'support', 'candle_size',
       'delta_filter', 'delta_filter2', 'upper_wick', 'lower_wick',
       'loop_number'],
      dtype='object')

# Create and save best params df

In [66]:
curs = ['AUD_CAD',
        'AUD_CHF',
        'AUD_NZD',
        'AUD_USD',
        'CAD_CHF',
        'EUR_AUD',
        'EUR_CAD',
        'EUR_CHF',
        'EUR_USD',
        'GBP_USD',
        'NZD_CAD',
        'NZD_CHF',
        'NZD_USD',
        'USD_CAD',
        'USD_CHF'
        
        ]
final_df = pd.DataFrame([0])
for cur in curs:
    print(cur)
    file = 'Macro/SIM_CLEAN_MA_' + cur + '_M5_2016-01-01_2022-01-31.csv'
    print(file)
    df = pd.read_csv(file)
    print(df.shape)

        
    if final_df.shape[0] == 1:
        final_df = df
    else:
        final_df = final_df.append(df)
print()
print()
print()
print('FINAL_SHAPE',final_df.shape)
best_trade_df = pd.DataFrame([0])
best_ending_vals = []
final_lst = []
for cur in curs:
    

    buy = final_df[(final_df['instrument'] == cur) & (final_df['buy_or_sell'] == 'buy') & 
                   (final_df['bullish_ma'] == True) & (final_df['bullish_candle'] == False) & 
                   (final_df['actual_number_of_trades'] >= 75)].sort_values(by = 'ending_val',ascending = False).head(1)

    sell = final_df[(final_df['instrument'] == cur) & (final_df['buy_or_sell'] == 'sell') & 
                   (final_df['bullish_ma'] == False) & (final_df['bullish_candle'] == True) & 
                   (final_df['actual_number_of_trades'] >= 75)].sort_values(by = 'ending_val',ascending = False).head(1)
    
    best_ending_vals.append(buy['ending_val'].iloc[0])
    best_ending_vals.append(sell['ending_val'].iloc[0])
    
    final_lst.append([buy['instrument'].iloc[0],buy['ending_val'].iloc[0],buy['actual_number_of_trades'].iloc[0]])
    final_lst.append([sell['instrument'].iloc[0],sell['ending_val'].iloc[0],sell['actual_number_of_trades'].iloc[0]])

    if best_trade_df.shape[0] == 1:
        best_trade_df = buy
        best_trade_df = best_trade_df.append(sell)

    else:
        best_trade_df = best_trade_df.append(buy)
        best_trade_df = best_trade_df.append(sell)
        
        
param_cols = ['instrument','buy_or_sell','ending_val','sl', 'tp','lookup_range',
       'bullish_ma', 'bullish_candle', 'support', 'candle_size',
       'delta_filter', 'delta_filter2', 'upper_wick', 'lower_wick','RR','win_rate','actual_number_of_trades']
param_df = best_trade_df[param_cols]
file = 'best_trade_params.csv'
param_df.to_csv(file,index = False)

AUD_CAD
Macro/SIM_CLEAN_MA_AUD_CAD_M5_2016-01-01_2022-01-31.csv
(97650, 23)
AUD_CHF
Macro/SIM_CLEAN_MA_AUD_CHF_M5_2016-01-01_2022-01-31.csv
(63126, 23)
AUD_NZD
Macro/SIM_CLEAN_MA_AUD_NZD_M5_2016-01-01_2022-01-31.csv


<ipython-input-66-8eedbffc5330>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
<ipython-input-66-8eedbffc5330>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)


(49787, 23)
AUD_USD
Macro/SIM_CLEAN_MA_AUD_USD_M5_2016-01-01_2022-01-31.csv
(64764, 23)


<ipython-input-66-8eedbffc5330>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
<ipython-input-66-8eedbffc5330>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)


CAD_CHF
Macro/SIM_CLEAN_MA_CAD_CHF_M5_2016-01-01_2022-01-31.csv
(67788, 23)
EUR_AUD
Macro/SIM_CLEAN_MA_EUR_AUD_M5_2016-01-01_2022-01-31.csv
(98406, 23)
EUR_CAD
Macro/SIM_CLEAN_MA_EUR_CAD_M5_2016-01-01_2022-01-31.csv


...

<ipython-input-66-8eedbffc5330>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)


USD_CHF
Macro/SIM_CLEAN_MA_USD_CHF_M5_2016-01-01_2022-01-31.csv
(48132, 23)


<ipython-input-66-8eedbffc5330>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)





FINAL_SHAPE (956231, 23)


<ipython-input-66-8eedbffc5330>:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_trade_df = best_trade_df.append(sell)
<ipython-input-66-8eedbffc5330>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_trade_df = best_trade_df.append(buy)
<ipython-input-66-8eedbffc5330>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_trade_df = best_trade_df.append(sell)
<ipython-input-66-8eedbffc5330>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  best_trade_df = best_trade_df.append(buy)
<ipython-input-66-8eedbffc5330>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

In [47]:
file = 'best_trade_params.csv'
param_df = pd.read_csv(file)
print(param_df.shape)
param_df.head()

(30, 14)


,instrument,buy_or_sell,ending_val,sl,tp,lookup_range,bullish_ma,bullish_candle,support,candle_size,delta_filter,delta_filter2,upper_wick,lower_wick
0,AUD_CAD,buy,92757.5,0.00125,0.0090,500,True,False,True,0.00050,0.0020,0.0005,0.0000,0.0000
1,AUD_CAD,sell,206865.5,0.00125,0.0090,1000,False,True,False,0.00005,0.0005,0.0015,0.0000,0.0000
2,AUD_CHF,buy,52802.5,0.00050,0.0015,750,True,False,False,0.00150,0.0005,0.0070,0.0000,0.0000
3,AUD_CHF,sell,143577.0,0.00125,0.0090,500,False,True,True,0.00050,0.0015,0.0070,0.0000,0.0004
4,AUD_NZD,buy,245188.0,0.00075,0.0090,500,True,False,True,0.00075,0.0005,0.0070,0.0002,0.0000
